These are some basic utility functions we will need. 

In [54]:
from __future__ import division
from collections import Counter, defaultdict
import os, math, random, re, glob #Imports a bunch of modules we will need
import pandas, requests, json
import sklearn
import sklearn.feature_extraction.text

Some functions for splitting data

In [2]:
def split_data(data, prob):
    """split data into fractions [prob, 1 - prob]"""
    results = [], []
    for row in data:
        results[0 if random.random() < prob else 1].append(row)
    return results

def train_test_split(x, y, test_pct):
    data = zip(x, y)                              # pair corresponding values ; zip combines lists to tuples
    train, test = split_data(data, 1 - test_pct)  # split the dataset of pairs
    x_train, y_train = zip(*train)                # magical un-zip trick
    x_test, y_test = zip(*test)
    return x_train, x_test, y_train, y_test

# NAIVE BAYES

## Naive Bayes from Scratch

First, let's build a Naive Bayes classifier from scratch. This example drawn from *Data Science from Scratch* by Joel Grus. 

### Mathematical Preliminaries

Recall the key independence assumption of Naive Bayes: $P(X_1 = x_1,\dots,X_n = x_n\,|\,S) = P(X_1 = x_1\,|\,S)\times \dots 
    \times P(X_n = x_n\,|\,S)$

To be concrete, let's assume we are building a spam filter. 

Given a vocabulary $w_1,\dots,w_n$, let $X_i$ be the event "message contains $w_i$." $X_i = x_i, x_i \in \{0,1\}$. 

$S$ is the event "message is spam" and $\neg S$ is the event "message is not spam."

According to Bayes' Theorem 

$P(S\,|\,X_1 = x_1,\dots, X_n = x_n) = \frac{P(X_1 = x_1,\dots, X_n = x_n\,|\,S)P(S)}{P(X_1 = x_1,\dots, X_n = x_n)} = \frac{P(X_1 = x_1,\dots, X_n = x_n\,|\,S)P(S)}{P(X_1 = x_1,\dots, X_n = x_n\,|\,S)P(S)\, + \,P(X_1 = x_1,\dots, X_n = x_n\,|\,\neg S)P(\neg S)}$

We further assume that we have no knowledge of the prior probability of spam; so $P(S) = P(\neg S) = 0.5$ (this is the principle of indifference)

With this simplification, $P(S\,|\,X_1 = x_1,\dots, X_n = x_n) = \frac{P(X_1 = x_1,\dots, X_n = x_n\,|\,S)}{P(X_1 = x_1,\dots, X_n = x_n\,|\,S)\, +\, P(X_1 = x_1,\dots, X_n = x_n\,|\,\neg S)}$ 

Now we make the Naive Bayes assumption: $P(X_1 = x_1,\dots,X_n = x_n\,|\,S) = P(X_1 = x_1\,|\,S)\times \dots 
    \times P(X_n = x_n\,|\,S)$
    
We can estimate $P(X_i = x_i\,|\,S)$ by computing the fraction of spam messages containing the word $i$, e.g., Obamacare. 

Smoothing: $P(X_i\,|\,S) = \frac{(k + \textrm{number of spams containing}\, w_i)}{(2k + \textrm{number of spams})}$



### Now we are going to code this up

First we will "tokenize" our input, i.e., turn text into presence/absence of words. 

In [3]:
def tokenize(message): #Note: no stemming
    message = message.lower() # convert to lowercase
    all_words = re.findall("[a-z0-9']+", message) # use re to extract the words
    return set(all_words) # remove duplicates because we are creating a set

Now we need to count the number of times each word shows up in spam and non-spam

In [4]:
def count_words(training_set):
    """training set consists of pairs (message, is_spam)""" 
    counts = defaultdict(lambda: [0, 0]) #This is a tricky bit of Python magic, makes a dictionary initialized to [0,0]
    for message, is_spam in training_set: #Here I step through the training set.
        for word in tokenize(message): 
            counts[word][0 if is_spam else 1] += 1
    return counts

We need a function to convert these counts into (smoothed) probabilities

In [5]:
def word_probabilities(counts, total_spams, total_non_spams, k=0.5): #What is the smoothing parameter?
    """turn the word_counts into a list of triplets 
    w, p(w | spam) and p(w | ~spam)"""
    return [(w,
             (spam + k) / (total_spams + 2 * k),
             (non_spam + k) / (total_non_spams + 2 * k)) #This uses list comprehension. 
             for w, (spam, non_spam) in counts.items()]  #Replace .iteritems with .items for Python3

Now we need to come up with a way to compute the spam probability for a message, given word probabilities. With the Naive Bayes assumption, we *would* be multiplying together a bunch of probabilities. This is bad (underflow) so we compute:

$p_1 *\dots*p_n = \exp(\, \log(p_1) + \dots + \log(p_n)\,)$; recall $\log(ab) = \log a + \log b$ and $\exp(\, \log x \,) = x$

Thank you, John Napier (1550-1617)

In [6]:
def spam_probability(word_probs, message):
    message_words = tokenize(message)
    log_prob_if_spam = log_prob_if_not_spam = 0.0 #Initialize; we are working with log probs to deal with underflow.

    for word, prob_if_spam, prob_if_not_spam in word_probs: #We iterate over all possible words we've observed

        # for each word in the message, 
        # add the log probability of seeing it 
        if word in message_words:
            log_prob_if_spam += math.log(prob_if_spam) #This is prob of seeing word if spam
            log_prob_if_not_spam += math.log(prob_if_not_spam) #This is prob of seeing word if not spam

        # for each word that's not in the message
        # add the log probability of _not_ seeing it
        else:
            log_prob_if_spam += math.log(1.0 - prob_if_spam)
            log_prob_if_not_spam += math.log(1.0 - prob_if_not_spam)
            
    prob_if_spam = math.exp(log_prob_if_spam) #Compute numerator
    prob_if_not_spam = math.exp(log_prob_if_not_spam)
    return prob_if_spam / (prob_if_spam + prob_if_not_spam) #Compute whole thing and return

Think: how would this change if $P(S) \neq P(\neg S)$

Now we write a class (this is a Python term) for our Naive Bayes Classifier

In [7]:
class NaiveBayesClassifier:

    def __init__(self, k=0.5):
        self.k = k
        self.word_probs = [] #Initializes word_probs as an empty list, sets a default smoothing parameters

    def train(self, training_set): #Operates on the training_set
    
        # count spam and non-spam messages: first step of training
        num_spams = len([is_spam 
                         for message, is_spam in training_set 
                         if is_spam]) #This is also list comprehension
        num_non_spams = len(training_set) - num_spams

        # run training data through our "pipeline"
        word_counts = count_words(training_set)
        self.word_probs = word_probabilities(word_counts, 
                                             num_spams, 
                                             num_non_spams,
                                             self.k) #"Train" classifier 
                                             
    def classify(self, message):
        return spam_probability(self.word_probs, message) #Now we have all we need to classify a message


We'll need a special utility function for reading in the data

In [11]:
def get_subject_data(path):

    data = []

    # regex for stripping out the leading "Subject:" and any spaces after it
    subject_regex = re.compile(r"^Subject:\s+")

    # glob.glob returns every filename that matches the wildcarded path
    for fn in glob.glob(path):
        is_spam = "ham" not in fn
        
        #with open(fn,'r') as file: #PYTHON 3 USERS: COMMENT THIS OUT AND USE LINE BELOW
        with open(fn,'r',errors='surrogateescape') as file: 
            for line in file:
                if line.startswith("Subject:"):
                    subject = subject_regex.sub("", line).strip()
                    data.append((subject, is_spam))

    return data

Grab data from: https://spamassassin.apache.org/publiccorpus/

Get the ones with 20021010 prefixes and put them into the data folder under the current working directory.

In [9]:
path = os.getcwd() + '/data/*/*/*'

In [12]:
data = get_subject_data(path)

In [13]:
len(data) #How many 

3423

In [14]:
data[1000] #Let's look

('Re: The 3rd Annual Consult Hyperion Digital Identity Forum', False)

To train the model, we'll need to split our data into training & test

In [15]:
random.seed(0) #This is important for replicability
train_data,test_data = split_data(data,0.75) #Recall what the second argument does here

In [16]:
classifier = NaiveBayesClassifier() #Create an instance of our classifier

In [17]:
classifier.train(train_data) #Train the classifier on the training data

In [18]:
len(train_data)

2547

Some simple evaluation:

In [19]:
# triplets (subject, actual is_spam, predicted spam probability)
classified = [(subject, is_spam, classifier.classify(subject))
              for subject, is_spam in test_data]

# assume that spam_probability > 0.5 corresponds to spam prediction # and count the combinations of (actual is_spam, predicted is_spam)
counts = Counter((is_spam, spam_probability > 0.5) # (actual, predicted)
                     for _, is_spam, spam_probability in classified)

In [20]:
counts #Let's see how we did!

Counter({(False, False): 704,
         (False, True): 33,
         (True, False): 38,
         (True, True): 101})

In [21]:
precision = counts[(True,True)]/(counts[(False,True)]+counts[(True,True)]) #True positives over all positive predictions
print(precision)

0.753731343283582


In [22]:
recall = counts[(True,True)]/(counts[(True,False)]+counts[(True,True)])#what fraction of positives identified
print(recall)

0.7266187050359713


**Think**: what will happen if I change my spam threshold from 0.5?

**Think**: what decision rule are we using, when we assign the class label $\hat{y} = S$ when $P(S\,|\,X_1\dots X_n) = \frac{P(S)\prod_i P(X_i = x_i|S)}{P(\textbf{X})} > 0.5$

**Think**: what will happen if I split the data differently (e.g., less training data, more testing data). Try it!

We can also find words that lead to a high probability of spam (using Bayes' Theorem):

In [23]:
def p_spam_given_word(word_prob):
    """uses bayes's theorem to compute p(spam | message contains word)"""
    # word_prob is one of the triplets produced by word_probabilities

    word, prob_if_spam, prob_if_not_spam = word_prob
    return prob_if_spam / (prob_if_spam + prob_if_not_spam)

In [24]:
words = sorted(classifier.word_probs,key=p_spam_given_word)

In [25]:
spammiest_words = words[-15:]
hammiest_words = words[:15]

In [26]:
spammiest_words

[('account', 0.01780821917808219, 0.00022893772893772894),
 ('norton', 0.02054794520547945, 0.00022893772893772894),
 ('per', 0.02054794520547945, 0.00022893772893772894),
 ('29', 0.02054794520547945, 0.00022893772893772894),
 ('95', 0.02054794520547945, 0.00022893772893772894),
 ('mortgage', 0.023287671232876714, 0.00022893772893772894),
 ('guaranteed', 0.023287671232876714, 0.00022893772893772894),
 ('adv', 0.026027397260273973, 0.00022893772893772894),
 ('zzzz', 0.026027397260273973, 0.00022893772893772894),
 ('clearance', 0.026027397260273973, 0.00022893772893772894),
 ('year', 0.028767123287671233, 0.00022893772893772894),
 ('sale', 0.031506849315068496, 0.00022893772893772894),
 ('rates', 0.031506849315068496, 0.00022893772893772894),
 ('systemworks', 0.036986301369863014, 0.00022893772893772894),
 ('money', 0.03972602739726028, 0.00022893772893772894)]

In [27]:
hammiest_words

[('spambayes', 0.0013698630136986301, 0.04601648351648352),
 ('users', 0.0013698630136986301, 0.036401098901098904),
 ('razor', 0.0013698630136986301, 0.030906593406593408),
 ('zzzzteana', 0.0013698630136986301, 0.029075091575091576),
 ('sadev', 0.0013698630136986301, 0.026785714285714284),
 ('ouch', 0.0013698630136986301, 0.02358058608058608),
 ('bliss', 0.0013698630136986301, 0.021749084249084248),
 ('selling', 0.0013698630136986301, 0.021291208791208792),
 ('apt', 0.0013698630136986301, 0.021291208791208792),
 ('wedded', 0.0013698630136986301, 0.021291208791208792),
 ('perl', 0.0013698630136986301, 0.0190018315018315),
 ('spamassassin', 0.0013698630136986301, 0.018543956043956044),
 ('satalk', 0.00410958904109589, 0.053800366300366304),
 ('problem', 0.0013698630136986301, 0.017170329670329672),
 ('alsa', 0.0013698630136986301, 0.016712454212454212)]

# Let' Try it on Our Clinton Obama Corpora

In [41]:
def getGithubFiles(target, maxFiles = 100):
    #We are setting a max so our examples don't take too long to run
    #For converting to a DataFrame
    releasesDict = {
        'name' : [], #The name of the file
        'text' : [], #The text of the file, watch out for binary files
        'path' : [], #The path in the git repo to the file
        'html_url' : [], #The url to see the file on Github
        'download_url' : [], #The url to download the file
    }

    #Get the directory information from Github
    r = requests.get(target)

    #Check for rate limiting
    if r.status_code != 200:
        raise RuntimeError("Github didn't like your request, you have probably been rate limited.")
    filesLst = json.loads(r.text)

    for fileDict in filesLst[:maxFiles]:
        #These are provided by the directory
        releasesDict['name'].append(fileDict['name'])
        releasesDict['path'].append(fileDict['path'])
        releasesDict['html_url'].append(fileDict['html_url'])
        releasesDict['download_url'].append(fileDict['download_url'])

        #We need to download the text though
        text = requests.get(fileDict['download_url']).text
        releasesDict['text'].append(text)

    return pandas.DataFrame(releasesDict)

'''
#Uncomment this to download your own data

targetSenator = 'Obama'# = ['Voinovich', 'Obama', 'Whitehouse', 'Snowe', 'Rockefeller', 'Murkowski', 'McCain', 'Kyl', 'Baucus', 'Frist']

ObamaReleases = pandas.DataFrame()

print("Fetching {}'s data".format(targetSenator))
targetDF = getGithubFiles('https://api.github.com/repos/lintool/GrimmerSenatePressReleases/contents/raw/{}'.format(targetSenator), maxFiles = 2000)
targetDF['targetSenator'] = targetSenator
ObamaReleases = ObamaReleases.append(targetDF, ignore_index = True)

targetSenator = 'Clinton'# = ['Voinovich', 'Obama', 'Whitehouse', 'Snowe', 'Rockefeller', 'Murkowski', 'McCain', 'Kyl', 'Baucus', 'Frist']

ClintonReleases = pandas.DataFrame()

print("Fetching {}'s data".format(targetSenator))
targetDF = getGithubFiles('https://api.github.com/repos/lintool/GrimmerSenatePressReleases/contents/raw/{}'.format(targetSenator), maxFiles = 2000)
targetDF['targetSenator'] = targetSenator
ClintonReleases = ClintonReleases.append(targetDF, ignore_index = True)

ObamaClintonReleases = ObamaReleases.append(ClintonReleases, ignore_index = True)
ObamaClintonReleases.to_csv("data/ObamaClintonReleases.csv")

'''



#senReleasesTraining = pandas.read_csv("data/senReleasesTraining.csv")


Fetching Obama's data
Fetching Clinton's data


In [53]:
ObamaClintonReleases = pandas.read_csv("data/ObamaClintonReleases.csv")
ObamaClintonReleases = ObamaClintonReleases.dropna(axis=0, how='any')

Let's turn the 'targetSenator' column into a binary variable.

In [81]:
def IsObama(targetSenator):
    if targetSenator == 'Obama':
        isObama = True
    elif targetSenator == 'Clinton':
        isObama = False
    return(isObama)

In [82]:
ObamaClintonReleases['IsObama'] = ObamaClintonReleases['targetSenator'].apply(lambda x: IsObama(x))

Let's split the data into training data and test data.

In [84]:
data = list(zip(ObamaClintonReleases['text'], ObamaClintonReleases['IsObama']))
random.seed(0) #This is important for replicability
train_data,test_data = split_data(data,0.75) #Recall what the second argument does here

In [85]:
print (len(train_data))
print (len(test_data))

1275
434


In [87]:
classifier = NaiveBayesClassifier() 
classifier.train(train_data) #Train the classifier on the training data

Some simple evaluation: